Importing the libraries

In [1]:
import numpy as np 
import pandas as pd 
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
restraunt_data = pd.read_csv('Dataset .csv')

In [3]:
restraunt_data.shape

(9551, 21)

In [4]:
restraunt_data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [5]:
restraunt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

Data Preprocessing 

In [6]:
#Deleting Unnnecessary Columns
restaurant = restraunt_data.drop(['Rating color','Restaurant ID','Currency'],axis =1)

In [7]:
#Removing the Duplicates 
restaurant.duplicated().sum()
restaurant.drop_duplicates(inplace=True)

In [8]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant Name       9551 non-null   object 
 1   Country Code          9551 non-null   int64  
 2   City                  9551 non-null   object 
 3   Address               9551 non-null   object 
 4   Locality              9551 non-null   object 
 5   Locality Verbose      9551 non-null   object 
 6   Longitude             9551 non-null   float64
 7   Latitude              9551 non-null   float64
 8   Cuisines              9542 non-null   object 
 9   Average Cost for two  9551 non-null   int64  
 10  Has Table booking     9551 non-null   object 
 11  Has Online delivery   9551 non-null   object 
 12  Is delivering now     9551 non-null   object 
 13  Switch to order menu  9551 non-null   object 
 14  Price range           9551 non-null   int64  
 15  Aggregate rating     

In [9]:
#Removing the NaN values if any 
restaurant.isnull().sum()
restaurant.dropna(how = 'any', inplace=True)
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9542 entries, 0 to 9550
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant Name       9542 non-null   object 
 1   Country Code          9542 non-null   int64  
 2   City                  9542 non-null   object 
 3   Address               9542 non-null   object 
 4   Locality              9542 non-null   object 
 5   Locality Verbose      9542 non-null   object 
 6   Longitude             9542 non-null   float64
 7   Latitude              9542 non-null   float64
 8   Cuisines              9542 non-null   object 
 9   Average Cost for two  9542 non-null   int64  
 10  Has Table booking     9542 non-null   object 
 11  Has Online delivery   9542 non-null   object 
 12  Is delivering now     9542 non-null   object 
 13  Switch to order menu  9542 non-null   object 
 14  Price range           9542 non-null   int64  
 15  Aggregate rating      9542

In [10]:
#reading column names 
restaurant.columns

Index(['Restaurant Name', 'Country Code', 'City', 'Address', 'Locality',
       'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Has Table booking', 'Has Online delivery',
       'Is delivering now', 'Switch to order menu', 'Price range',
       'Aggregate rating', 'Rating text', 'Votes'],
      dtype='object')

In [11]:
#changing the column names 
restaurant = restaurant.rename(columns = {'Average Cost for two':'cost','Restaurant Name':'Name'})
restaurant.columns

Index(['Name', 'Country Code', 'City', 'Address', 'Locality',
       'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'cost',
       'Has Table booking', 'Has Online delivery', 'Is delivering now',
       'Switch to order menu', 'Price range', 'Aggregate rating',
       'Rating text', 'Votes'],
      dtype='object')

In [12]:
restaurant['cost']

0       1100
1       1200
2       4000
3       1500
4       1500
        ... 
9546      80
9547     105
9548     170
9549     120
9550      55
Name: cost, Length: 9542, dtype: int64

In [13]:
#some transformations 
restaurant['cost'] = restaurant['cost'].astype(str)#changing the cost to string as if any value with '' is present is considered as string
restaurant['cost'] = restaurant['cost'].apply(lambda x:x.replace(',','.'))#Using lambda function to replace ',' from cost
restaurant['cost'] = restaurant['cost'].astype(float) # Changing the cost to Float
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9542 entries, 0 to 9550
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  9542 non-null   object 
 1   Country Code          9542 non-null   int64  
 2   City                  9542 non-null   object 
 3   Address               9542 non-null   object 
 4   Locality              9542 non-null   object 
 5   Locality Verbose      9542 non-null   object 
 6   Longitude             9542 non-null   float64
 7   Latitude              9542 non-null   float64
 8   Cuisines              9542 non-null   object 
 9   cost                  9542 non-null   float64
 10  Has Table booking     9542 non-null   object 
 11  Has Online delivery   9542 non-null   object 
 12  Is delivering now     9542 non-null   object 
 13  Switch to order menu  9542 non-null   object 
 14  Price range           9542 non-null   int64  
 15  Aggregate rating      9542

In [14]:
#Adjusting the column names
restaurant.Name = restaurant.Name.apply(lambda x:x.title())
restaurant['Has Table booking'].replace(('Yes','No'),(1, 0) , inplace = True)
restaurant['Has Online delivery'].replace(('Yes','No'),(1, 0) , inplace = True)
restaurant['Is delivering now'].replace(('Yes','No'),(1, 0) , inplace = True)
restaurant['Switch to order menu'].replace(('Yes','No'),(1, 0) , inplace = True)
restaurant.cost.unique()

/var/folders/3b/kzp02sjx6j35fsg93flz7p900000gn/T/ipykernel_28798/2395168427.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  restaurant['Has Table booking'].replace(('Yes','No'),(1, 0) , inplace = True)
/var/folders/3b/kzp02sjx6j35fsg93flz7p900000gn/T/ipykernel_28798/2395168427.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future

array([1.10e+03, 1.20e+03, 4.00e+03, 1.50e+03, 1.00e+03, 2.00e+03,
       6.00e+03, 8.00e+02, 9.00e+02, 7.00e+02, 8.50e+02, 6.00e+02,
       3.00e+03, 1.80e+03, 5.50e+01, 3.00e+01, 4.50e+01, 6.00e+01,
       5.00e+01, 8.00e+01, 9.00e+01, 2.00e+02, 1.50e+02, 1.00e+02,
       2.30e+02, 2.40e+02, 3.50e+02, 1.40e+02, 1.70e+02, 1.20e+02,
       2.50e+02, 4.00e+01, 7.00e+01, 3.00e+02, 8.50e+01, 7.50e+01,
       4.00e+02, 6.50e+01, 2.50e+01, 1.00e+01, 0.00e+00, 2.00e+01,
       3.50e+01, 7.00e+00, 2.70e+02, 5.00e+02, 4.30e+02, 3.15e+02,
       2.20e+02, 9.50e+01, 1.90e+02, 2.60e+02, 1.80e+02, 3.30e+02,
       2.80e+02, 1.60e+02, 2.85e+02, 1.30e+02, 1.10e+02, 2.50e+03,
       3.60e+03, 5.50e+02, 1.40e+03, 4.50e+02, 9.50e+02, 6.50e+02,
       1.30e+03, 2.40e+03, 1.60e+03, 2.20e+03, 7.50e+02, 1.70e+03,
       1.25e+03, 1.90e+03, 2.70e+03, 3.50e+03, 5.00e+03, 2.30e+03,
       1.75e+03, 3.20e+03, 2.10e+03, 1.85e+03, 4.50e+03, 3.30e+03,
       1.35e+03, 1.65e+03, 1.95e+03, 1.05e+03, 2.90e+02, 1.45e

In [15]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9542 entries, 0 to 9550
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  9542 non-null   object 
 1   Country Code          9542 non-null   int64  
 2   City                  9542 non-null   object 
 3   Address               9542 non-null   object 
 4   Locality              9542 non-null   object 
 5   Locality Verbose      9542 non-null   object 
 6   Longitude             9542 non-null   float64
 7   Latitude              9542 non-null   float64
 8   Cuisines              9542 non-null   object 
 9   cost                  9542 non-null   float64
 10  Has Table booking     9542 non-null   int64  
 11  Has Online delivery   9542 non-null   int64  
 12  Is delivering now     9542 non-null   int64  
 13  Switch to order menu  9542 non-null   int64  
 14  Price range           9542 non-null   int64  
 15  Aggregate rating      9542

In [16]:
restaurant.head()

,Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,cost,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating text,Votes
0,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100.0,1,0,0,0,3,4.8,Excellent,314
1,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200.0,1,0,0,0,3,4.5,Excellent,591
2,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,1,0,0,0,4,4.4,Very Good,270
3,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500.0,0,0,0,0,4,4.9,Excellent,365
4,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500.0,1,0,0,0,4,4.8,Excellent,229


In [17]:
restaurant['City'].unique()

array(['Makati City', 'Mandaluyong City', 'Pasay City', 'Pasig City',
       'Quezon City', 'San Juan City', 'Santa Rosa', 'Tagaytay City',
       'Taguig City', 'Bras�_lia', 'Rio de Janeiro', 'S��o Paulo',
       'Albany', 'Armidale', 'Athens', 'Augusta', 'Balingup',
       'Beechworth', 'Boise', 'Cedar Rapids/Iowa City', 'Chatham-Kent',
       'Clatskanie', 'Cochrane', 'Columbus', 'Consort', 'Dalton',
       'Davenport', 'Des Moines', 'Dicky Beach', 'Dubuque',
       'East Ballina', 'Fernley', 'Flaxton', 'Forrest', 'Gainesville',
       'Hepburn Springs', 'Huskisson', 'Inverloch', 'Lakes Entrance',
       'Lakeview', 'Lincoln', 'Lorn', 'Macedon', 'Macon', 'Mayfield',
       'Mc Millan', 'Middleton Beach', 'Monroe', 'Montville',
       'Ojo Caliente', 'Orlando', 'Palm Cove', 'Paynesville', 'Penola',
       'Pensacola', 'Phillip Island', 'Pocatello', 'Potrero', 'Princeton',
       'Rest of Hawaii', 'Savannah', 'Singapore', 'Sioux City',
       'Tampa Bay', 'Tanunda', 'Trentham East', '

In [18]:
# Compute Mean Rating
restaurant['MeanRating'] = restaurant.groupby('Name')['Aggregate rating'].transform('mean')


In [19]:
restaurant.head()

,Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,cost,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating text,Votes,MeanRating
0,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100.0,1,0,0,0,3,4.8,Excellent,314,4.8
1,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200.0,1,0,0,0,3,4.5,Excellent,591,4.5
2,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,1,0,0,0,4,4.4,Very Good,270,4.4
3,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500.0,0,0,0,0,4,4.9,Excellent,365,4.9
4,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500.0,1,0,0,0,4,4.8,Excellent,229,4.8


In [20]:
# Normalize Mean Rating using MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 5))
restaurant[['MeanRating']] = scaler.fit_transform(restaurant[['MeanRating']]).round(2)

Text Preprocessing 

Some of the common text preprocessing / cleaning steps are:

Lower casing
Removal of Punctuations
Removal of Stopwords
Removal of URLs
Spelling correction

In [21]:
restaurant['Rating text'].unique()

array(['Excellent', 'Very Good', 'Good', 'Average', 'Not rated', 'Poor'],
      dtype=object)

In [22]:
#lower casing 
restaurant['Cuisines'] = restaurant['Cuisines'].str.lower()

In [23]:
restaurant['Rating text'] = restaurant['Rating text'].str.lower()

In [24]:
restaurant['Name'] = restaurant['Name'].str.lower()

In [25]:
restaurant['Locality'] = restaurant['Locality'].str.lower()

In [26]:
restaurant['Address'] = restaurant['Address'].str.lower()

In [27]:
restaurant['City'] = restaurant['City'].str.lower()

In [28]:
restaurant[['Cuisines' ,'Rating text']].head()

,Cuisines,Rating text
0,"french, japanese, desserts",excellent
1,japanese,excellent
2,"seafood, asian, filipino, indian",very good
3,"japanese, sushi",excellent
4,"japanese, korean",excellent


In [29]:
#Removing Puncutation Marks if any 
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation))
restaurant['Cuisines'] = restaurant['Cuisines'].apply(lambda text:remove_punctuation(text))
restaurant['Rating text'] = restaurant['Rating text'].apply(lambda text:remove_punctuation(text))
restaurant['Name'] = restaurant['Name'].apply(lambda text:remove_punctuation(text))
restaurant['Locality'] = restaurant['Locality'].apply(lambda text:remove_punctuation(text))
restaurant['Address'] = restaurant['Address'].apply(lambda text:remove_punctuation(text))
restaurant['City'] = restaurant['City'].apply(lambda text:remove_punctuation(text))

In [30]:
restaurant[['Name','Locality','Address','City','Cuisines' , 'Rating text',]].head()


,Name,Locality,Address,City,Cuisines,Rating text
0,le petit souffle,century city mall poblacion makati city,third floor century city mall kalayaan avenue ...,makati city,french japanese desserts,excellent
1,izakaya kikufuji,little tokyo legaspi village makati city,little tokyo 2277 chino roces avenue legaspi v...,makati city,japanese,excellent
2,heat edsa shangrila,edsa shangrila ortigas mandaluyong city,edsa shangrila 1 garden way ortigas mandaluyon...,mandaluyong city,seafood asian filipino indian,very good
3,ooma,sm megamall ortigas mandaluyong city,third floor mega fashion hall sm megamall orti...,mandaluyong city,japanese sushi,excellent
4,sambo kojin,sm megamall ortigas mandaluyong city,third floor mega atrium sm megamall ortigas ma...,mandaluyong city,japanese korean,excellent


In [31]:
#Removal of Stopwords 
stopwords = set(stopwords.words('english'))
print(stopwords)

{'had', "she's", 'shan', 'against', "hadn't", 'but', 'her', 'most', 'there', 'doing', 'above', "shouldn't", "mightn't", 'why', 'should', 'being', 'by', "couldn't", 'they', 'our', 'does', 'them', 'after', 'doesn', 'any', 'aren', 'with', 'whom', 'other', 'just', 'yourself', "should've", 'd', 'when', 'its', 'i', 'themselves', 'hers', 'a', 'is', 'haven', "haven't", "wasn't", 'few', "doesn't", 'she', 'o', 'these', 'up', 'am', 'no', "don't", 'here', "you've", 'of', 'wouldn', 'did', 'has', 've', 'for', "aren't", 'won', 'ours', "hasn't", 'ain', 'shouldn', 'wasn', 'if', 'ourselves', 'this', 'at', 'some', 'don', "it's", 'off', 'all', 'weren', 'below', 'or', "wouldn't", 'been', 'each', 'because', 'very', "won't", 'an', 'during', 's', 'hadn', 'and', 'your', 'yourselves', 'that', 'under', 'needn', 'are', 'own', 'you', 'how', 'yours', 'theirs', 'on', 'further', 'isn', 'about', 'having', 'have', 'be', 'to', 'couldn', 'he', 'than', 'out', 'down', 'then', 'same', 'too', 'didn', 'itself', 'my', 'their',

In [32]:
def remove_stopwords(text):
     """custom function to remove the stopwords"""
     words = text.split()
     filtered_words = [word for word in words if word.lower not in stopwords]
     return ' '.join(filtered_words)
restaurant['Cuisines'] = restaurant['Cuisines'].apply(lambda text:remove_stopwords(text))
restaurant['Rating text'] = restaurant['Rating text'].apply(lambda text:remove_stopwords(text))

In [33]:
restaurant[['Cuisines' , 'Rating text']].head()

,Cuisines,Rating text
0,french japanese desserts,excellent
1,japanese,excellent
2,seafood asian filipino indian,very good
3,japanese sushi,excellent
4,japanese korean,excellent


In [34]:
restaurant['Name'].head()

0        le petit souffle
1        izakaya kikufuji
2    heat  edsa shangrila
3                    ooma
4             sambo kojin
Name: Name, dtype: object

In [37]:
(restaurant['Price range'].unique())

array([3, 4, 2, 1])

In [38]:
restaurant.shape

(9542, 19)

In [39]:
# Ordinal encoding for 'Price range'
price_mapping = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
restaurant['Price range (encoded)'] = restaurant['Price range'].map(price_mapping)

In [40]:
restaurant.head()

,Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,cost,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating text,Votes,MeanRating,Price range (encoded)
0,le petit souffle,162,makati city,third floor century city mall kalayaan avenue ...,century city mall poblacion makati city,"Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,french japanese desserts,1100.0,1,0,0,0,3,4.8,excellent,314,4.92,High
1,izakaya kikufuji,162,makati city,little tokyo 2277 chino roces avenue legaspi v...,little tokyo legaspi village makati city,"Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,japanese,1200.0,1,0,0,0,3,4.5,excellent,591,4.67,High
2,heat edsa shangrila,162,mandaluyong city,edsa shangrila 1 garden way ortigas mandaluyon...,edsa shangrila ortigas mandaluyong city,"Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,seafood asian filipino indian,4000.0,1,0,0,0,4,4.4,very good,270,4.59,Very High
3,ooma,162,mandaluyong city,third floor mega fashion hall sm megamall orti...,sm megamall ortigas mandaluyong city,"SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,japanese sushi,1500.0,0,0,0,0,4,4.9,excellent,365,5.00,Very High
4,sambo kojin,162,mandaluyong city,third floor mega atrium sm megamall ortigas ma...,sm megamall ortigas mandaluyong city,"SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,japanese korean,1500.0,1,0,0,0,4,4.8,excellent,229,4.92,Very High


In [41]:
# Applying TF-IDF Vectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.01, stop_words='english')
tfidf_matrix = tfidf.fit_transform(restaurant['Cuisines'])

In [42]:
#getting the similarity score using cosine similarityscore algorithms 
similarity = cosine_similarity(tfidf_matrix)

In [43]:
print(similarity)

[[1.         0.81887863 0.         ... 0.         0.         0.        ]
 [0.81887863 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]
 [0.         0.         0.         ... 0.         1.         1.        ]]


Restaurant recommendation system

In [54]:
# Function to get recommendations based on cosine similarity
Cuisine_name = input('Enter Cuisine You want to eat:')
list_of_all_cuisines = restaurant['Cuisines'].tolist()

# Finding closest match
find_close_match_cuisine = difflib.get_close_matches(Cuisine_name, list_of_all_cuisines)
close_match = find_close_match_cuisine[0]

# Find the index of the cuisine
restaurant_index = restaurant[restaurant['Cuisines'] == close_match].index[0]

# Get similarity scores for the selected restaurant
similarity_score = list(enumerate(similarity[restaurant_index]))

# Sort based on similarity score
sorted_similar_cuisines = sorted(similarity_score, key=lambda x: x[1], reverse=True)

# Output the top 30 recommended restaurants
print('Restaurants suggested for you:')
i = 1
for rest in sorted_similar_cuisines[:30]:
    index = rest[0]
    restaurant_name = restaurant[restaurant.index == index]['Name'].values[0]
    print(f"{i}. {restaurant_name}")
    i += 1


Enter Cuisine You want to eat: italian


Restaurants suggested for you:
1. villa tevere
2. gero
3. doc ristorante
4. paribar
5. terra�o it��lia
6. five  ten
7. chilis grill  bar
8. red ginger sushi grill  bar
9. catfish charlies
10. downtown grill
11. ohana
12. the crab shack
13. johnnie mars
14. bellabrava
15. edison fooddrink lab
16. 306 north restaurant
17. bleu cafe
18. barbeque nation
19. virgin courtyard
20. crispy crust
21. oh my
22. italia cafe by aura
23. yum yum cha
24. le marche sugar  spice cafe
25. captain bill deliverz
26. jalsa lounge
27. spice it  hotel ibis
28. shri ram restaurant
29. zaffran  the bristol hotel
30. stacks and racks
